# Garbage Collection

In [1]:
import torch as th

In [2]:
import syft as sy

In [3]:
hook = sy.TorchHook(th)

In [4]:
bob = sy.VirtualWorker(hook,id="bob")

In [5]:
x = th.tensor([1,2,3,4]).send(bob)

In [6]:
bob._objects

{37389800037: tensor([1, 2, 3, 4])}

In [7]:
del x

In [8]:
bob._objects

{}

In [9]:
for i in range(1000):
    x = th.tensor([1,2,3,4]).send(bob)

In [10]:
bob._objects

{46220452336: tensor([1, 2, 3, 4])}

In [11]:
del x

In [12]:
bob._objects

{}

# Toy Federated Learning

# Advanced Remote Execution

In [13]:
naufil = sy.VirtualWorker(hook, id="naufil")
faizan = sy.VirtualWorker(hook, id="faizan")

In [14]:
x = th.tensor([1,2,3,4]).send(naufil).send(faizan)

In [15]:
x

(Wrapper)>[PointerTensor | me:72092432281 -> faizan:376974663]

In [16]:
faizan._objects

{376974663: (Wrapper)>[PointerTensor | faizan:376974663 -> naufil:63528494901]}

In [17]:
naufil._objects

{63528494901: tensor([1, 2, 3, 4])}

In [18]:
del x

In [19]:
naufil._objects

{63528494901: tensor([1, 2, 3, 4])}

In [20]:
faizan._objects

{376974663: (Wrapper)>[PointerTensor | faizan:376974663 -> naufil:63528494901]}

In [21]:
x

NameError: name 'x' is not defined